# Importing the necessary libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

# Extraction using the 1st website

## Request to get accessed to the html of website

In [15]:
url_list_cheese = "https://www.fromages.com/fr/encyclopedie"
html_data = requests.get(url_list_cheese).text

KeyboardInterrupt: 

In [ ]:
soup_list_cheese = BeautifulSoup(html_data,"html.parser")

## Get the necessary tag elements where the information is contained

In [ ]:
list_of_letters = []
list_a_tags = []
soup_letter = soup_list_cheese.find_all('div',{'class':'lettre_unit'})
for element in soup_letter:
    list_of_letters.append(element.find_all('a'))

for letter in list_of_letters:
    for a_tag in letter:
        list_a_tags.append("https://www.fromages.com/" + a_tag.get("href"))

[<div class="lettre_unit">
<div class="lettre_titre color-orange" id="lettre_A">
                            A                        </div>
<ul>
<li>
<a href="/fr/fromage/551-abbaye-de-tamie">
                                        Abbaye de Tamié                                    </a>
</li>
<li>
<a href="/fr/fromage/588-a-casinca">
                                        A Casinca                                    </a>
</li>
<li>
<a href="/fr/fromage/22-abbaye-de-timadeuc">
                                        Abbaye de Timadeuc                                    </a>
</li>
<li>
<a href="/fr/fromage/11-abbaye-du-mont-des-cats">
                                        Abbaye du Mont des Cats                                    </a>
</li>
<li>
<a href="/fr/fromage/17-affidelice-au-chablis">
                                        Affidelice au Chablis                                    </a>
</li>
<li>
<a href="/fr/fromage/76-anneau-du-vic-bilh">
                                   

## Create different functions for each attribute you want to collect

In [ ]:
def get_nom_fromage(soup):
    soup_nom_fromage = soup.find_all('h1',{'class': 'fromage_detail_titre'})[0].text
    nom_fromage = [word for word in soup_nom_fromage.replace("\r\n","").replace("\n\n","").split(" ") if word != ""]
    if nom_fromage == []:
        return np.nan
    else:
        nom_fromage = ' '.join(nom_fromage)
        return nom_fromage

In [ ]:
def get_poids(soup):
    soup_poids = soup.find_all('div',{'class':'picto_poids'})[0].text
    poids = [word for word in soup_poids.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]
    if poids == []:
        return np.nan
    else:
        poids.pop(0)
        poids = ' '.join(poids)
        return poids

In [ ]:
def get_affinage(soup):
    soup_affinage = soup.find_all('div',{'class':'picto_affinage'})[0].text
    affinage = [word for word in soup_affinage.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]
    if affinage == []:
        return np.nan
    else:    
        affinage.pop(0)
        affinage = ' '.join(affinage)
        return affinage

In [ ]:
def get_type(soup):
    soup_type = soup.find_all('div',{'class':'picto_type'})[0].text
    type = [word for word in soup_type.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]
    if type == []:
        return np.nan
    else:
        type.pop(0)
        type = ' '.join(type)
        return type

In [ ]:
def get_lait(soup):
    soup_lait = soup.find_all('div',{'class':'picto_lait'})[0].text
    lait = [word for word in soup_lait.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]
    if lait == []:
        return np.nan
    else:
        lait = ' '.join(lait)
        return lait

In [ ]:
def get_pate(soup):
    soup_pate = soup.find_all('div',{'class':'picto_pate'})[0].text
    pate = [word for word in soup_pate.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]

    if pate == []:
        return np.nan
    else:
        pate.pop(0)
        pate = ' '.join(pate)
        return pate

In [ ]:
def get_saison(soup):
    soup_saison = soup.find_all('div',{'class':'picto_saison'})[0].text
    saison = [word for word in soup_saison.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]

    if saison == []:
        return np.nan
    else:
        saison.pop(0)
        saison = ' '.join(saison)
        saison = re.sub(r"([A-Z])", r" \1", saison).split()
        saison = ' '.join(saison)
        return saison

In [ ]:
def get_vin(soup):
    soup_vin = soup.find_all('p',{'class':'fromage_detail_vins_liste'})

    if soup_vin == []:
        return np.nan
    else:
        vin = [word for word in soup_vin[0].text.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]
        vin = ' '.join(vin)
        return vin

In [ ]:
def get_mariage_fromage(soup):
    soup_mariage_fromage = soup.find_all('ul',{'class':'fromage_detail_vins_liste'})

    if soup_mariage_fromage == []:
        return np.nan
    else:
        mariage_fromage = [word for word in soup_mariage_fromage[0].text.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]
        mariage_fromage = ' '.join(mariage_fromage)
        return mariage_fromage

In [ ]:
def get_ingredients(soup):
    soup_ingredients = soup.find_all('div',{'class':'col-xs-12 font-light'})

    if soup_ingredients == []:
        return np.nan
    else:
        ingredients = [word for word in soup_ingredients[0].text.replace("\r\n","").replace("\n\n","").replace("\n","").replace(":","").split(" ") if word != ""]
        ingredients.pop(0)
        if ingredients.count("/") > 0:
            ingredients = ingredients[:ingredients.index("/")]
        ingredients = ' '.join(ingredients)
        return ingredients

## Create the dataframe

In [ ]:
df_cheese = pd.DataFrame(columns=["nom","poids","affinage","type","type_lait","pate","saison","mariage_vin","mariage_fromage","ingredients"])

In [ ]:
for url in list_a_tags:
    data = requests.get(url).text
    soup = BeautifulSoup(data,"html.parser")
    nom_fromage = get_nom_fromage(soup)
    poids = get_poids(soup)
    affinage = get_affinage(soup)
    type = get_type(soup)
    lait = get_lait(soup)
    pate = get_pate(soup)
    saison = get_saison(soup)
    vin = get_vin(soup)
    mariage_fromage = get_mariage_fromage(soup)
    ingredients = get_ingredients(soup)

    df_cheese = df_cheese.append(
        {"nom":nom_fromage,
         "poids":poids,
         "affinage":affinage,
         "type":type,
         "type_lait":lait,
         "pate":pate,
         "saison":saison,
         "mariage_vin":vin,
         "mariage_fromage":mariage_fromage,
         "ingredients":ingredients}
        ,ignore_index=True)
    
df_cheese

,nom,poids,affinage,type,type_lait,pate,saison,mariage_vin,mariage_fromage,ingredients
0,Abbaye de Tamié,"510 g (17,85 Ozs)",2 moisMinimum,à la pièce,Laitde vache,pressées non cuites,Printemps Automne Hiver,Vin rouge: Beaujolais-Villages (cépage: Gamay ...,NaN,"lait de vache, ferments lactiques, présure, sel"
1,A Casinca,400 g (14 Ozs),,à la coupe,Laitde chèvre,molles à croûte lavée,Printemps Eté Automne Hiver,grand vin corse blanc patrimonio,NaN,NaN
2,Abbaye de Timadeuc,200 g (7 Ozs),3 semaines minimum,à la coupe,Laitde vache,pressées non cuites,Printemps Eté Automne,NaN,NaN,NaN
3,Abbaye du Mont des Cats,200 g (7 Ozs),2 mois minimum,à la coupe,Laitde vache,pressées non cuites,Printemps Eté Automne,"Vin rouge: Graves (cépages: Merlot, Cabernet-S...",NaN,NaN
4,Affidelice au Chablis,"250 g (8,75 Ozs)",3 à 4 semaines,à la pièce,Laitde vache,molles à croûte lavée,Printemps Automne Hiver,Vin blanc très sec: Chablis (cépage: Chardonnay),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
339,Valencay AOP,"220 g (7,7 Ozs)",Minimum 5 semaines,à la pièce,Laitde chèvre,molle,Printemps Eté Automne,"Vin blanc sec: Valençay (cépages: Sauvignon, C...",Sainte Maure de Touraine AOP,"Lait cru de chèvre, sel, ferments lactiques, p..."
340,Venaco,200 g (7 Ozs),2 moisMinimum,à la coupe,Laitde brebis,molles à croûte lavée,Printemps Eté Automne Hiver,NaN,NaN,NaN
341,Vieux Lille,"187 g (6,545 Ozs)",3 moisMinimum,à la coupe,Laitde vache,molles à croûte lavée,Printemps Eté Automne Hiver,Champagne,NaN,NaN
342,Voeu du Poitou,"150 g (5,25 Ozs)",Minimum 2 semaines,à la coupe,Laitde chèvre,molle,Printemps Eté Automne,NaN,NaN,NaN


In [ ]:
df_cheese['type_lait'] = df_cheese['type_lait'].str.replace("Laitde ","  ").replace('Lait','vache')
df_cheese['saison'] = df_cheese.saison.apply(lambda x: x[0:len(x)].split(' '))

In [ ]:
df_cheese

,nom,poids,affinage,type,type_lait,pate,saison,mariage_vin,mariage_fromage,ingredients
0,Abbaye de Tamié,"510 g (17,85 Ozs)",2 moisMinimum,à la pièce,vache,pressées non cuites,"[Printemps, Automne, Hiver]",Vin rouge: Beaujolais-Villages (cépage: Gamay ...,NaN,"lait de vache, ferments lactiques, présure, sel"
1,A Casinca,400 g (14 Ozs),,à la coupe,chèvre,molles à croûte lavée,"[Printemps, Eté, Automne, Hiver]",grand vin corse blanc patrimonio,NaN,NaN
2,Abbaye de Timadeuc,200 g (7 Ozs),3 semaines minimum,à la coupe,vache,pressées non cuites,"[Printemps, Eté, Automne]",NaN,NaN,NaN
3,Abbaye du Mont des Cats,200 g (7 Ozs),2 mois minimum,à la coupe,vache,pressées non cuites,"[Printemps, Eté, Automne]","Vin rouge: Graves (cépages: Merlot, Cabernet-S...",NaN,NaN
4,Affidelice au Chablis,"250 g (8,75 Ozs)",3 à 4 semaines,à la pièce,vache,molles à croûte lavée,"[Printemps, Automne, Hiver]",Vin blanc très sec: Chablis (cépage: Chardonnay),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
339,Valencay AOP,"220 g (7,7 Ozs)",Minimum 5 semaines,à la pièce,chèvre,molle,"[Printemps, Eté, Automne]","Vin blanc sec: Valençay (cépages: Sauvignon, C...",Sainte Maure de Touraine AOP,"Lait cru de chèvre, sel, ferments lactiques, p..."
340,Venaco,200 g (7 Ozs),2 moisMinimum,à la coupe,brebis,molles à croûte lavée,"[Printemps, Eté, Automne, Hiver]",NaN,NaN,NaN
341,Vieux Lille,"187 g (6,545 Ozs)",3 moisMinimum,à la coupe,vache,molles à croûte lavée,"[Printemps, Eté, Automne, Hiver]",Champagne,NaN,NaN
342,Voeu du Poitou,"150 g (5,25 Ozs)",Minimum 2 semaines,à la coupe,chèvre,molle,"[Printemps, Eté, Automne]",NaN,NaN,NaN


In [ ]:
df_cheese.isnull().sum()

nom                  0
poids                0
affinage             0
type                 0
type_lait            0
pate                 0
saison               0
mariage_vin        101
mariage_fromage    281
ingredients        216
dtype: int64

# Extraction using the 2nd website

The following information should be added for each type of cheese: <br>
- Ingredients
- Nutritional Information: Details like fat content, calorie count, protein content, etc., might be important to some users.
<br>

Change the following information:
- Features like weight (poids) might need to be normalized so they're on a similar scale to your other features.


In [ ]:
# Using this link, we notice that there is a table containing most of the information that we wanted our cheese dataframe to have.
#We can just use the read_html function from pandas to get the table and then merge it with our dataframe.

#check if you can add more information to this table by scraping the website & check with the others what contains the most information
url_list_cheese = "https://www.quiveutdufromage.com/liste-des-fromages"
table = pd.read_html(url_list_cheese)
additional_cheese_information = table[0]

In [ ]:
additional_cheese_information

,Fromage,Type de lait Tout voir lait de vache lait de chèvre Lait de brebis lait de bufflonne Lait de brebis et de chèvre,Origine & régions Tout voir Bourgogne Franche Comté Auvergne-Rhône-Alpes Suisse Italie Nouvelle-Aquitaine Provence Alpes Côte d'azur Hauts de France Occitanie Pays de la Loire Grand-Est Île de France Centre-Val de Loire Angleterre Corse Espagne Normandie Pays-Bas Grèce Belgique Bretagne,Type de pâte Tout voir Pâte molle à croûte lavée Pâte pressée cuite Pâte fraîche Pâte pressée non cuite Pâte molle à croûte fleurie Fromages de chèvre Pâte persillée Pâte pressée Pâte molle,Goût Tout voir Fruité Doux mi-fort Intense Équilibré
0,Abbaye de Cîteaux,lait de vache,Bourgogne Franche Comté,Pâte molle à croûte lavée,Fruité
1,Abbaye de Tamié,lait de vache,Auvergne-Rhône-Alpes,Pâte molle à croûte lavée,Fruité
2,Abondance AOP,lait de vache,Auvergne-Rhône-Alpes,Pâte pressée cuite,Fruité
3,Apérivrais®,lait de vache,Auvergne-Rhône-Alpes,Pâte fraîche,Doux
4,Appenzeller,lait de vache,Suisse,Pâte pressée non cuite,Fruité
...,...,...,...,...,...
190,Trèfle du Perche,lait de chèvre,Centre-Val de Loire,Fromages de chèvre,Doux
191,Vacherin fribourgeois,lait de vache,Suisse,Pâte pressée non cuite,Fruité
192,Valençay AOP,lait de chèvre,Centre-Val de Loire,Fromages de chèvre,Doux
193,Vieux Pané®,lait de vache,Pays de la Loire,Pâte molle à croûte lavée,Fruité


In [ ]:
#First we will change the columns of the dataframe
additional_cheese_information.columns = ['nom', 'type_lait', 'origine', 'type_pate', 'gout']

In [ ]:
additional_cheese_information

,nom,type_lait,origine,type_pate,gout
0,Abbaye de Cîteaux,lait de vache,Bourgogne Franche Comté,Pâte molle à croûte lavée,Fruité
1,Abbaye de Tamié,lait de vache,Auvergne-Rhône-Alpes,Pâte molle à croûte lavée,Fruité
2,Abondance AOP,lait de vache,Auvergne-Rhône-Alpes,Pâte pressée cuite,Fruité
3,Apérivrais®,lait de vache,Auvergne-Rhône-Alpes,Pâte fraîche,Doux
4,Appenzeller,lait de vache,Suisse,Pâte pressée non cuite,Fruité
...,...,...,...,...,...
190,Trèfle du Perche,lait de chèvre,Centre-Val de Loire,Fromages de chèvre,Doux
191,Vacherin fribourgeois,lait de vache,Suisse,Pâte pressée non cuite,Fruité
192,Valençay AOP,lait de chèvre,Centre-Val de Loire,Fromages de chèvre,Doux
193,Vieux Pané®,lait de vache,Pays de la Loire,Pâte molle à croûte lavée,Fruité


# Extraction on 3rd Website

In [3]:
# Collect information from this website [Fromage Encyclopédie](https://www.laboitedufromager.com/liste-des-fromages-par-ordre-alphabetique/) and compare it to the other dataframes
# If you can add information to the other dataframes to complete list do it, this goes the same way for the other dataframes
# If you can't add information to the other dataframes, choose the one with the most valuable information
url_list_cheese_site_3 = "https://www.laboitedufromager.com/liste-des-fromages-par-ordre-alphabetique/"
html_data_site_3 = requests.get(url_list_cheese_site_3).text

In [4]:
soup_list_cheese_site_3 = BeautifulSoup(html_data_site_3,"html.parser")
soup_list_cheese_site_3

<!DOCTYPE html>
<html dir="ltr" lang="fr-FR" prefix="og: https://ogp.me/ns#"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="ie=edge" http-equiv="X-UA-Compatible"/><link href="https://www.laboitedufromager.com/wp-content/themes/laboitedufromager/src/images/favicon.png" rel="shortcut icon" type="image/png"/> <script defer="" src="data:text/javascript;base64,ZG9jdW1lbnQuZG9jdW1lbnRFbGVtZW50LmNsYXNzTmFtZSA9IGRvY3VtZW50LmRvY3VtZW50RWxlbWVudC5jbGFzc05hbWUgKyAnIHllcy1qcyBqc19hY3RpdmUganMn"></script> <title>Fromage : La liste (presque) exhaustive des fromages français</title><meta content="Retrouvez ici le classement des fromages par ordre alphabétique : Le fromage : L'essentiel à connaître Devenu un incontournable dans le monde culinaire, le fromage existe dans une grande variété selon les goûts, la texture et les saveurs. Si au début, il était réservé aux classes populaires et aux montagnards, aujourd’hui, il s’invite à tout

In [ ]:
#Find the table in the dataframe
tables = soup_list_cheese_site_3.find_all('table')

#Seperate the name of the cheeses, the family and crust

#Create a list of the a tags
a_tags = []
for table in tables:
    a_tags.append(table.find_all('a'))

#Find the href in the list of a tags
href = []
for a_tag in a_tags:
    for element in a_tag:
        if "https://www.laboitedufromager.com" not in element.get("href"):
            href.append("https://www.laboitedufromager.com" + element.get("href"))
        else:
            href.append(element.get("href"))

cheese_name = []
cheese_family = []
cheese_crust = []

#Since the table has some cheese that don't have links, we will have to filter them out
for table in tables:
    rows = table.find_all('tr')
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 1:
            if cells[0].text in ' '.join(str(e.text) for e in a_tags[0]) and cells[0].text not in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
                cheese_name.append(cells[0].text)
                cheese_family.append(cells[1].text)
                cheese_crust.append(cells[2].text)
            if cells[0].text == "Vacherin Fribourgeois":
                cheese_name.append(cells[0].text)
                cheese_family.append(cells[1].text)
                cheese_crust.append(cells[2].text)

In [ ]:
# Use the href to get the information of the cheeses

#Origin of cheese
def provenance(soup):
    provenance = []
    ul = soup.find('ul', {'class':'product_attributes'})
    if ul is None:
        provenance = np.nan
    else:
        li = ul.find_all('li')
        provenance = li[0].text
    return provenance

#Price of cheese - I don't know if you need to include this as the price changes everytime (look for another solution)
# def prix(soup):
#     div_product_details = soup.find('div', {'class':'woocommerce-product-details__short-description product_excerpt'})
#     p = div_product_details.find_all('p')[-1]
#     prix = p.text.replace("€","").replace("\xa0","")
#     return prix

#The different senses of that cheese (sight, touch, nose, touch)
def voyage_sensoriel(soup):
    li = soup.find('li', {'id':'voyage'})
    if li is None:
        list_senses = [np.nan, np.nan, np.nan, np.nan]
    else:
        lis = li.find_all('li')
        list_senses = []
        for i in lis:
            list_senses.append(i.text.replace("\xa0"," "))
    return list_senses

#The different pairings that can come with the cheese
def accords(soup):
    li = soup.find('li', {'id':'accords'})
    if li is None:
        accords = np.nan
    else:
        accords = li.find_all('p')
    return accords

#The similar cheeses we have in this category
def similar_cheeses(soup):
    h2 = soup.find_all('h2', {'class':'woocommerce-loop-product__title'})
    similar_cheeses_names = [name.text for name in h2]
    return similar_cheeses_names

cheeses = pd.DataFrame(columns=["name","milk","crust","origin", "senses", "pairings", "similarities"])

for index in range(0,len(href)):
    data = requests.get(href[index]).text
    soup = BeautifulSoup(data,"html.parser")
    name = cheese_name[index]
    milk = cheese_family[index]
    crust = cheese_crust[index]
    origin = provenance(soup)
    senses = voyage_sensoriel(soup)
    pairings = accords(soup)
    similarities = similar_cheeses(soup) 
    
    cheeses = cheeses.append(
        {
            "name": name,
            "milk": milk,
            "crust":crust,
            "origin": origin,
            "senses": senses,
            "pairings": pairings,
            "similarities": similarities
        }, 
        ignore_index = True
    )

cheeses.to_csv("cheese.csv", index = False, encoding="utf-8")

,name,milk,crust,origin,senses,pairings,similarities
0,Abondance,Vache,Pressée cuite,Auvergne_Rhône-AlpesLabel,[Oeil : Croûtage lisse présentant les marques ...,[[L’Abondance se consomme généralement avec un...,"[Gruyère Suisse d’Alpage AOP 12 Mois, Parmesan..."
1,Angors,Chèvre,Molle à croûte naturelle,Nouvelle AquitaineLait,"[Oeil : forme de Losange, Nez : Odeur légèreme...","[[L’Angors s’accompagne d’un vin blanc sec, te...","[Crottin de Chavignol AOP, Bleu de Chèvre, Sai..."
2,Anneau de Saint Clément,Chèvre,Molle à croûte naturelle,Nouvelle AquitaineLait,"[Œil : Croûte striée, cendrée. Pâte blanche, s...",[[L’anneau de Saint-Clément s’accompagne d’un ...,"[Bouton de Culotte fermier, Crottin de Chavign..."
3,Anneau du Vic-Bilh,Chèvre,Molle à croûte naturelle,OccitanieLait,[Oeil : Croûte légèrement fleurie et plissée. ...,[[L’anneau de vic-bilh s’accompagne d’un vin b...,"[Valençay AOP fermier, Bouton de Culotte fermi..."
4,Appenzeller,Vache,Pressée cuite,SuisseLait,"[Oeil : croûte lisse et collante, Nez : Odeur ...",[[L’Appenzeller se marie à un vin blanc sec te...,"[Etivaz AOP, Tête de Moine entière, Gruyère Su..."
...,...,...,...,...,...,...,...
314,Venise Verte,Vache,Molle à croûte lavée,Pays de la Loire% Matière Grasse/Poids Total,"[Œil : croûte orangée, Nez : Odeur soutenue, G...",[[La Venise Verte s’accompagne de vin blanc fr...,"[Maroilles fermier AOP 1/2, Bergues au torchon..."
315,Vercorais,Vache,Pressée cuite,Auvergne_Rhône-AlpesProducteur,"[Oeil : Pâte jaune pressée cuite., Gout : Lége...",[[Le formage Vercorais ? On le déguste finemen...,"[Morbier AOP 100jours, Herve à la Bière, Parme..."
316,Vieux Lille,Vache,Molle à croûte lavée,Ferme du Château CourbetProvenance,"[Oeil : Croûte orangé homogène, lisse et brill...",[[Le Vieux Lille se mange surtout en fin de re...,"[Abondance AOP, Beaufort chalet alpage AOP, To..."
317,Villageois blanc,Chèvre,Molle à croûte naturelle,Nouvelle AquitaineProducteur,"[Oeil : Pâte molle à croûte naturelle., Gout :...",[[La dégustation optimale d’un fromage se fait...,"[Pouligny-Saint-Pierre AOP fermier, Chabichou ..."


In [3]:
cheese_df = pd.read_csv("C:\\Users\\Sascha\\OneDrive\\Desktop\\Projects\\FromageFinder\\cheese.csv")
cheese_df

,name,milk,crust,origin,senses,pairings,similarities
0,Abondance,Vache,Pressée cuite,Provenance : Auvergne_Rhône-AlpesLabel : Appel...,['Oeil : Croûtage lisse présentant les marques...,[<p>L’Abondance se consomme généralement avec ...,"['Gruyère Suisse d’Alpage AOP 12 Mois', 'Parme..."
1,Angors,Chèvre,Molle à croûte naturelle,Provenance : Nouvelle AquitaineLait : Lait cru...,"['Oeil : forme de Losange', 'Nez : Odeur légèr...","[<p>L’Angors s’accompagne d’un vin blanc sec, ...","['Crottin de Chavignol AOP', 'Bleu de Chèvre',..."
2,Anneau de Saint Clément,Chèvre,Molle à croûte naturelle,Provenance : Nouvelle AquitaineLait : Lait cru...,"['Œil : Croûte striée, cendrée. Pâte blanche, ...",[<p>L’anneau de Saint-Clément s’accompagne d’u...,"['Bouton de Culotte fermier', 'Crottin de Chav..."
3,Anneau du Vic-Bilh,Chèvre,Molle à croûte naturelle,Provenance : OccitanieLait : Lait cru de chèvr...,['Oeil : Croûte légèrement fleurie et plissée....,[<p>L’anneau de vic-bilh s’accompagne d’un vin...,"['Valençay AOP fermier', 'Bouton de Culotte fe..."
4,Appenzeller,Vache,Pressée cuite,Provenance : SuisseLait : Lait cru de vache% M...,"['Oeil : croûte lisse et collante', 'Nez : Ode...","[<p style=""text-align: justify;"">L’Appenzeller...","['Etivaz AOP', 'Tête de Moine entière', 'Gruyè..."
...,...,...,...,...,...,...,...
314,Venise Verte,Vache,Molle à croûte lavée,Provenance : Pays de la Loire% Matière Grasse/...,"['Œil : croûte orangée', 'Nez : Odeur soutenue...",[<p>La Venise Verte s’accompagne de vin blanc ...,"['Maroilles fermier AOP 1/2', 'Bergues au torc..."
315,Vercorais,Vache,Pressée cuite,Provenance : Auvergne_Rhône-AlpesProducteur : ...,"['Oeil : Pâte jaune pressée cuite.', 'Gout : L...",[<p>Le formage Vercorais ? On le déguste finem...,"['Morbier AOP 100jours', 'Herve à la Bière', '..."
316,Vieux Lille,Vache,Molle à croûte lavée,Producteur : Ferme du Château CourbetProvenanc...,"['Oeil : Croûte orangé homogène, lisse et bril...","[<p style=""text-align: justify;"">Le Vieux Lill...","['Abondance AOP', 'Beaufort chalet alpage AOP'..."
317,Villageois blanc,Chèvre,Molle à croûte naturelle,Provenance : Nouvelle AquitaineProducteur : Fr...,"['Oeil : Pâte molle à croûte naturelle.', 'Gou...",[<p>La dégustation optimale d’un fromage se fa...,"['Pouligny-Saint-Pierre AOP fermier', 'Chabich..."


### Data cleaning

In [4]:
#Steps to take origin from the origin column
#Replace the nan values with an empty string
#Create a list of the origin column
#Iterate over the list
#Check if the value contains the word "Provenance : "
#If it does, split the string by the word "Provenance : "
#If it doesn't, append an empty string to the list
#Create a new column with the list
#Delete the origin column

cheese_df['origin'] = cheese_df['origin'].fillna("")
new_origin = []
origin_list = list(cheese_df['origin'])
for index in range(0,len(origin_list)):
    if "Provenance : " in origin_list[index]:
        new_origin.append(origin_list[index].split("Provenance : ")[1].replace("Label","").replace('Lait',"").replace('Producteur',"").replace("Particularité","").replace('%',":").split(":")[0])
    else:
        new_origin.append("")

new_origin

['Auvergne_Rhône-Alpes ',
 'Nouvelle Aquitaine ',
 'Nouvelle Aquitaine ',
 'Occitanie ',
 'Suisse ',
 'Occitanie ',
 'Pays de la Loire',
 'Auvergne_Rhône-Alpes ',
 'Auvergne_Rhône-Alpes ',
 'Hauts-de-France',
 'Auvergne_Rhône-Alpes ',
 'Occitanie ',
 'Occitanie',
 'Occitanie',
 'Auvergne_Rhône-Alpes ',
 'Auvergne_Rhône-Alpes ',
 'Occitanie ',
 'Auvergne_Rhône-Alpes',
 'Auvergne_Rhône-Alpes, Bourgogne - Franche-Comté ',
 'Auvergne_Rhône-Alpes ',
 'Auvergne_Rhône-Alpes ',
 'Auvergne_Rhône-Alpes',
 'Auvergne_Rhône-Alpes ',
 'Occitanie ',
 'Auvergne_Rhône-Alpes',
 'Auvergne_Rhône-Alpes ',
 'Auvergne_Rhône-Alpes',
 'Normandie ',
 'Auvergne_Rhône-Alpes ',
 'Bourgogne - Franche-Comté ',
 'Bourgogne - Franche-Comté',
 'Occitanie',
 'Auvergne_Rhône-Alpes ',
 '',
 '',
 'Ile de France ',
 'Ile de France ',
 'Ile de France',
 'Bourgogne - Franche-Comté ',
 'Bourgogne - Franche-Comté, Normandie ',
 'Corse',
 'Occitanie ',
 'Nouvelle Aquitaine ',
 'Occitanie ',
 'Centre-Val de Loire ',
 '',
 'Auverg

In [5]:
cheese_df['origin'] = new_origin
cheese_df

,name,milk,crust,origin,senses,pairings,similarities
0,Abondance,Vache,Pressée cuite,Auvergne_Rhône-Alpes,['Oeil : Croûtage lisse présentant les marques...,[<p>L’Abondance se consomme généralement avec ...,"['Gruyère Suisse d’Alpage AOP 12 Mois', 'Parme..."
1,Angors,Chèvre,Molle à croûte naturelle,Nouvelle Aquitaine,"['Oeil : forme de Losange', 'Nez : Odeur légèr...","[<p>L’Angors s’accompagne d’un vin blanc sec, ...","['Crottin de Chavignol AOP', 'Bleu de Chèvre',..."
2,Anneau de Saint Clément,Chèvre,Molle à croûte naturelle,Nouvelle Aquitaine,"['Œil : Croûte striée, cendrée. Pâte blanche, ...",[<p>L’anneau de Saint-Clément s’accompagne d’u...,"['Bouton de Culotte fermier', 'Crottin de Chav..."
3,Anneau du Vic-Bilh,Chèvre,Molle à croûte naturelle,Occitanie,['Oeil : Croûte légèrement fleurie et plissée....,[<p>L’anneau de vic-bilh s’accompagne d’un vin...,"['Valençay AOP fermier', 'Bouton de Culotte fe..."
4,Appenzeller,Vache,Pressée cuite,Suisse,"['Oeil : croûte lisse et collante', 'Nez : Ode...","[<p style=""text-align: justify;"">L’Appenzeller...","['Etivaz AOP', 'Tête de Moine entière', 'Gruyè..."
...,...,...,...,...,...,...,...
314,Venise Verte,Vache,Molle à croûte lavée,Pays de la Loire,"['Œil : croûte orangée', 'Nez : Odeur soutenue...",[<p>La Venise Verte s’accompagne de vin blanc ...,"['Maroilles fermier AOP 1/2', 'Bergues au torc..."
315,Vercorais,Vache,Pressée cuite,Auvergne_Rhône-Alpes,"['Oeil : Pâte jaune pressée cuite.', 'Gout : L...",[<p>Le formage Vercorais ? On le déguste finem...,"['Morbier AOP 100jours', 'Herve à la Bière', '..."
316,Vieux Lille,Vache,Molle à croûte lavée,Hauts-de-France,"['Oeil : Croûte orangé homogène, lisse et bril...","[<p style=""text-align: justify;"">Le Vieux Lill...","['Abondance AOP', 'Beaufort chalet alpage AOP'..."
317,Villageois blanc,Chèvre,Molle à croûte naturelle,Nouvelle Aquitaine,"['Oeil : Pâte molle à croûte naturelle.', 'Gou...",[<p>La dégustation optimale d’un fromage se fa...,"['Pouligny-Saint-Pierre AOP fermier', 'Chabich..."


In [111]:
senses_list = list(cheese_df.senses)

In [112]:
# Seperate each sense into its own list
senses_list = list(cheese_df.senses)
senses_list = list(map(lambda x: x.replace('Œil','Oeil'),senses_list))
senses_list = list(map(lambda x: x.replace('[','').replace(']',''),senses_list))
senses_list = list(map(lambda x: x.replace('Goût', 'Gout'), senses_list))

oeil = []
nez = []
gout = []
toucher = []

for i in senses_list:
    senses = i.split('\'')
    for sense in senses:
        if 'Oeil' in sense:
            oeil.append(sense.split(':')[1])
        if 'Nez' in sense:
            nez.append(sense.split(':')[1])
        if 'Gout' in sense:
            gout.append(sense.split(':')[1])
        if 'Toucher' in sense:
            toucher.append(sense.split(':')[1])

In [113]:
pairings_list = list(cheese_df.pairings)

In [121]:
pairings_list

['[<p>L’Abondance se consomme généralement avec un vin de Savoie. Par exemple, le Chasselas, un vin léger, ou alors un vin blanc sec : Chautagne blanc, Seyssel, Roussette du Bugey, Roussette de Savoie, Bugey blanc… Et pourquoi pas un Côtes du Jura blanc, un Anjou blanc ou un Bourgogne Vézelay ?</p>, <p>Par ailleurs, bonne nouvelle pour les amateurs de vin rouge, car l’Abondance se marie également avec ce nectar : Chignin rouge, Bourgogne Côte Chalonnaise rouge, Coteaux du Loir rouge… il y a l’embarres du choix.</p>]',
 '[<p>L’Angors\xa0s’accompagne d’un vin blanc sec, tel qu’un Charentais.</p>]',
 '[<p>L’anneau de Saint-Clément s’accompagne d’un vin blanc sec.</p>]',
 '[<p>L’anneau de vic-bilh s’accompagne d’un vin blanc de type Pacherenc.</p>]',
 '[<p style="text-align: justify;">L’Appenzeller se marie à un vin blanc sec tel qu’un Chablis, un muscadet ou un vin d’Alsace tel qu’un Pinot gris.</p>]',
 '[<p style="text-align: justify;">L’Ariégeois est un fromage qui s’harmonise très bien